<a href="https://colab.research.google.com/github/marcos-pd/IA_Prompting_CoderHouse/blob/main/Entrega_Final_CoderHouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generación de contenidos de difusión a partir de informes técnicos**
*Marcos Ferrrario*

*La solución que vamos a construir permite cargar un informe en pdf y mediante herramientas y librerías en Python extraer el texto que contiene. Ese texto se guardará en en una variable y se pasará a un modelo LLM para que, mediante un prompt optimizado, el modelo devuelva un texto con engagement que pueda usarse para su difusión mediante RS, email y publicación en web institucional.*


# *EXTRACIÓN DE TEXTO desde PDF*

Para la extracción de PDFs, las bibliotecas más comunes y robustas en Python son PyPDF2 (para PDFs basados en texto) y pdfplumber (que maneja mejor la estructura y tablas, y es más potente).

No es nuestro caso, pero para resolver el problema de pdfs escaneados, hay que usar una herramienta de OCR, donde PyTesseract es una buena opción.

In [1]:
# Instalar bibliotecas necesarias (ejecutar solo una vez por sesión)
!pip install PyPDF2
!pip install pdfplumber
!pip install google-colab

from google.colab import files
import pdfplumber
import re # Para futuras extracciones numéricas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00


In [2]:
# Subir el archivo PDF
print("Por favor, selecciona y sube tu archivo PDF (solo uno):")
uploaded = files.upload()

# Validamos la subida del archivo. Funciona mentras subas un solo archivo.
if uploaded:
    pdf_file_path = list(uploaded.keys())[0]
    print(f"Archivo '{pdf_file_path}' subido exitosamente.")
else:
    pdf_file_path = None
    print("No se ha subido ningún archivo.")

Por favor, selecciona y sube tu archivo PDF (solo uno):


Saving 202505 Informe Mensual SIOMA-ALLARIA.pdf to 202505 Informe Mensual SIOMA-ALLARIA.pdf
Archivo '202505 Informe Mensual SIOMA-ALLARIA.pdf' subido exitosamente.


In [3]:
def extract_info_from_pdf_pdfplumber(pdf_path):
    full_text = ""
    extracted_tables = []

    if not pdf_path:
        print("Error: No se proporcionó una ruta de archivo PDF válida.")
        return None, None

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                # Extraer texto de la página
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text + "\n\n"

                # Extraer tablas de la página
                tables = page.extract_tables()
                if tables:
                    extracted_tables.extend(tables)
    except Exception as e:
        print(f"Error al procesar el PDF '{pdf_path}': {e}")
        return None, None

    return full_text, extracted_tables

In [9]:
# Verificando la extracción
if 'pdf_file_path' in locals() and pdf_file_path:
    print(f"\nIntentando extraer información de: {pdf_file_path}")
    extracted_text_pdfplumber, extracted_tables_pdfplumber = extract_info_from_pdf_pdfplumber(pdf_file_path)

    if extracted_text_pdfplumber is not None:
        print("\n--- Texto extraído con pdfplumber (primeros 1000 caracteres) ---")
        print(extracted_text_pdfplumber[:1000])
        print("\n--- Fin del fragmento de texto ---")

        if len(extracted_text_pdfplumber) > 1000:
            print(f"\n(El texto completo tiene {len(extracted_text_pdfplumber)} caracteres. Se mostraron los primeros 1000.)")

        if extracted_tables_pdfplumber:
            print(f"\n--- Se encontraron {len(extracted_tables_pdfplumber)} tabla(s) ---")
            for i, table in enumerate(extracted_tables_pdfplumber):
                print(f"\nTabla {i+1} (primeras 3 filas):")
                for row_idx, row in enumerate(table):
                    print(row)
                    if row_idx >= 2:
                        break
                if i >= 0: # Para mostrar solo la primera tabla.
                    break
        else:
            print("\n--- No se encontraron tablas con pdfplumber. ---")
    else:
        print("\n--- La extracción de PDF falló o no se pudo obtener texto/tablas. ---")
else:
    print("\nError: La ruta del archivo PDF no está disponible. Asegurate de haber subido el archivo  y de que la variable 'pdf_file_path' se haya creado correctamente.")


Intentando extraer información de: 202505 Informe Mensual SIOMA-ALLARIA.pdf



--- Texto extraído con pdfplumber (primeros 1000 caracteres) ---
INFORME MENSUAL MACRO Y SECTORIAL
miércoles, 4 de junio de 2025
Novedades Macro:
Contexto: Argentina empezó en dic-23 un plan Inflación (apreciación). Ahora bien, de la mano
de estabilización cuyo objetivo es bajar la de la apreciación cambiaria, se suele observar
inflación de manera duradera para lograr un empeoramiento del déficit de cuenta
crecimiento de la economía estable y de largo corriente, lo que vuelve relevante conseguir
plazo. Según algunos trabajos de investigación financiamiento externo.
(*), estos planes tienen éxito solo en el 25% de En aquellos programas exitosos (Brasil 1994,
los casos. Las características de los planes de Chile 1989, Colombia 1998, Israel 1991, México
estabilización exitosos fueron, el superávit 1994 y Perú 1993) la baja de inflación vino
fiscal sostenido en el tiempo, un tipo de cambio acompañada con un fuerte crecimiento de la
nominal que se estabiliza y sube menos que la economía (P

# *PROCESAMIENTO DEL TEXTO EXTRAIDO a través del modelo OpenAI gpt-3.5-turbo*

In [10]:
from google.colab import userdata
from openai import OpenAI

# Carga la clave de API desde "los secretos" de Colab
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Inicializa el cliente de OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# Prueba una pequeña solicitud para verificar la conexión
try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", # "gpt-4"
        messages=[{"role": "user", "content": "Hola, ¿estás funcionando?"}]
    )
    print("Conexión a OpenAI exitosa:", response.choices[0].message.content)
except Exception as e:
    print(f"Error al conectar con OpenAI: {e}")
    print("Asegúrate de que tu clave de API esté configurada correctamente en los secretos de Colab y que tengas saldo disponible.")

Conexión a OpenAI exitosa: ¡Hola! Sí, estoy funcionando. ¿En qué puedo ayudarte hoy?


In [14]:
prompt_messages = [
    {"role": "system", "content": "Eres economista con muy buen conocimento de la macroeconomía argentina y un fuerte expertice en el mercado automotor. Tu diferencial como analista es tu capacidad para entender las implicancias de las novedades macroeconómicas en el negocio automotriz, desde la industria (fábricas, autopartistas o importadores) hasta el retail minorista (la venta de vehículos nuevos y usados al consumidor final). En tu rol, tienes el objetivo de analizar los informes técnicos que se te proveeran y a partir de ellos generar contenido para los canales de difusión de la empresa que los promueve. Ese contenido está orientado a Redes Sociales como Linkedin y a textos con engadgement para difundir los informes por email o en el sitio web de la empresa. Para generar esos contenidos, es importante que mantengas ciertas cualidades del lenguaje humano a través de las siguientes premisas: 1) Manteniendo la estructura general del texto y el rigor técnico, agregando emociones sutiles, expresiones humanas y lenguaje conversacional. 2) Redactar el texto con frases de ritmo variado, mezclando oraciones cortas con oraciones largas. 3) Sustituye palabras genéricas por otras más específicas que contengan carga emocional. 4) Agrega conectores suaves y naturales como “además”, “y es que”, etc. En la medida de lo posible usa ejemplos, analogías o detalles que hagan más cercano el texto al lector. 5) Evita estructuras gramaticales repetitivas, frases simétricas o lenguaje neutro."},
    {"role": "user", "content": f"""
    Aquí tienes un informe técnico detallado. Tu tarea es analizarlo para:

    1.  **Resumir el informe:** Extrae las 3 a 5 ideas principales o conclusiones clave.
    2.  **Identificar los hallazgos más importantes:** Aquellos que tienen mayor impacto o relevancia para el negocio automotriz.
    3.  **Generar el texto para una publicación de LinkedIn:** Esta publicación debe ser concisa, atractiva, profesional y resaltar los aspectos más relevantes del informe para una audiencia interesada en la cadena de valor automotriz en Argentina. Incluye un call to action (llamada a la acción) si aplica (ej. "¡Deja tus comentarios!", "¡Descubre más!"). Utiliza hashtags relevantes y evita emojis excesivos.

    ---
    **INFORME TÉCNICO:**
    {extracted_text_pdfplumber}

    ---
    """}
]

# Si se extrajeron datos numéricos clave, habría que añadirlos al prompt user así:
# if datos_numericos_str:
#     prompt_messages[1]["content"] += f"""
#     **DATOS NUMÉRICOS CLAVE DEL INFORME:**
#     {datos_numericos_str}
#     ---
#     """

print("Prompt generado (fragmento del mensaje del usuario):")
print(prompt_messages[1]["content"][:1000]) # Muestra los primeros 1000 caracteres

Prompt generado (fragmento del mensaje del usuario):

    Aquí tienes un informe técnico detallado. Tu tarea es analizarlo para:

    1.  **Resumir el informe:** Extrae las 3 a 5 ideas principales o conclusiones clave.
    2.  **Identificar los hallazgos más importantes:** Aquellos que tienen mayor impacto o relevancia para el negocio automotriz.
    3.  **Generar el texto para una publicación de LinkedIn:** Esta publicación debe ser concisa, atractiva, profesional y resaltar los aspectos más relevantes del informe para una audiencia interesada en la cadena de valor automotriz en Argentina. Incluye un call to action (llamada a la acción) si aplica (ej. "¡Deja tus comentarios!", "¡Descubre más!"). Utiliza hashtags relevantes y evita emojis excesivos.

    ---
    **INFORME TÉCNICO:**
    INFORME MENSUAL MACRO Y SECTORIAL
miércoles, 4 de junio de 2025
Novedades Macro:
Contexto: Argentina empezó en dic-23 un plan Inflación (apreciación). Ahora bien, de la mano
de estabilización cuyo objet

In [15]:
# Probamos eligiendo el modelo. 'gpt-3.5-turbo' ya que es más económico y rápido. Probamos también eligiendo 'gpt-4o' que es más caro, pero también más potente.
# model_name = "gpt-3.5-turbo"
model_name = "gpt-4o"

print(f"\nEnviando solicitud a OpenAI con el modelo {model_name}. Esto puede tomar unos segundos...")

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt_messages,
        temperature=0.5, # Controla la creatividad del modelo (0.0 = predecible, 1.0 = muy creativo)
        max_tokens=5000, # Límite en la longitud de la respuesta generada
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    generated_content = response.choices[0].message.content
    print("\n--- Contenido generado por OpenAI ---")
    print(generated_content)


except Exception as e:
    print(f"\nError al generar contenido con OpenAI: {e}")
    # Errores comunes: API Key incorrecta, límite de tokens excedido, modelo no accesible, problemas de facturación.
    # Revisa los detalles del error para depurar.


Enviando solicitud a OpenAI con el modelo gpt-4o. Esto puede tomar unos segundos...

--- Contenido generado por OpenAI ---
### Resumen del Informe:

1. **Plan de Estabilización Económica:** Argentina está implementando un plan de estabilización económica desde diciembre de 2023, con el objetivo de reducir la inflación y estabilizar la economía a largo plazo. Sin embargo, solo el 25% de estos planes suelen ser exitosos, y dependen de un superávit fiscal sostenido y un tipo de cambio estable.
   
2. **Reservas Internacionales y Financiamiento:** A pesar de la disminución de reservas en abril, el financiamiento externo ha permitido un aumento neto de las reservas en lo que va del año. Acuerdos con el FMI y el Banco Mundial han sido claves en este proceso.

3. **Mercado Cambiario y Tipo de Cambio:** El tipo de cambio oficial se ha mantenido estable, y se espera que continúe dentro de un rango controlado si se mantienen los ingresos de dólares esperados.

4. **Inflación y Tasas de Interés:

[PROXIMOS PASOS]

1) Afinar el prompting para optimizar el resultado
2) Generar imágenes para ilustar las publicaciones en Linkedin a partir de Gemini o Dall-E